# Bert Model

### 1. Import Packages, function et charghement donnees

#### 1.1 Import libraries

In [1]:
# libraries standard
import numpy as np
import pandas as pd
import random
import re

# dataviz
import seaborn as sns
import matplotlib.pyplot as plt

# ml et data handling
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler, TensorDataset, random_split

# BERT
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

# Experiment tracking
import mlflow
import mlflow.pytorch

#### 1.2 Functions

In [2]:
def accuracy(preds, labels):
    """
    Calcule la précision des prédictions.

    Args:
        preds: Les prédictions du modèle.
        labels: Les étiquettes réelles.

    Returns:
        float: La précision des prédictions.

    """
    pred_flat = np.argmax(preds, axis=1).flatten()
    label_flat = labels.flatten()
    return np.sum(pred_flat == label_flat) / len(label_flat)


def evaluate(dataloader_test):
    """
    Évalue les performances du modèle sur un ensemble de données de test.

    Args:
        dataloader_test: Le DataLoader contenant les données de test.

    Returns:
        float: La perte moyenne sur l'ensemble de données de test.
        numpy.array: Les prédictions du modèle.
        numpy.array: Les étiquettes réelles.

    """
    model.eval()
    loss_val_total = 0
    predictions, true_vals = [], []
    for batch in dataloader_test:
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        with torch.no_grad():
            outputs = model(**inputs)
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    loss_val_avg = loss_val_total / len(dataloader_test)
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    return loss_val_avg, predictions, true_vals

In [3]:
def Sentiment(sent):
    """
    Détermine le sentiment d'une phrase donnée en utilisant un modèle BERT pré-entraîné.

    Args:
        sent (str): La phrase pour laquelle le sentiment doit être déterminé.

    Returns:
        int: L'indice du sentiment prédit (0 pour négatif, 1 pour positif).
    """

    tokenizer = BertTokenizer.from_pretrained(output_dir)
    model_loaded = BertForSequenceClassification.from_pretrained(output_dir)
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens=True,
                        max_length=64,
                        pad_to_max_length=True,
                        return_attention_mask=True,
                        return_tensors='pt',
                   )

    input_id = encoded_dict['input_ids']

    attention_mask = encoded_dict['attention_mask']
    input_id = torch.LongTensor(input_id)
    attention_mask = torch.LongTensor(attention_mask)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_loaded = model_loaded.to(device)
    input_id = input_id.to(device)
    attention_mask = attention_mask.to(device)

    with torch.no_grad():
        outputs = model_loaded(input_id, token_type_ids=None, attention_mask=attention_mask)

    logits = outputs[0]
    index = logits.argmax()
    return index

#### 1.3 Charghement des donnees

In [4]:
# creation variable data
path = '../data/processed/'

In [5]:
# import des fichiers
data = pd.read_csv(path + 'data.csv')

### 2. Preparation donnees

In [6]:
# Échantillon de tweets
sample_data = data[['text', 'target']].sample(n=20000, random_state=42)

In [7]:
# Récupération des étiquettes et du texte à partir de l'échantillon de données
labels = sample_data.target.values
text = sample_data.text.values

In [8]:
# Séparation des données en ensembles d'entraînement et de test
df_train, df_testVal = train_test_split(sample_data, test_size=3200, shuffle=True)

# Affichage des tailles des ensembles d'entraînement et de test
print("Taille de l'ensemble d'entraînement:", len(df_train))
print("Taille de l'ensemble de test:", len(df_testVal))

Taille de l'ensemble d'entraînement: 16800
Taille de l'ensemble de test: 3200


In [9]:
# Séparation de l'ensemble de test en ensembles de test et de validation
df_test, df_val = train_test_split(df_testVal, test_size=0.5, shuffle=True)

# Affichage des tailles des ensembles de test et de validation
print("Taille de l'ensemble de test:", len(df_test))
print("Taille de l'ensemble de validation:", len(df_val))

Taille de l'ensemble de test: 1600
Taille de l'ensemble de validation: 1600


### 3. Modelisation

#### 3.1 Tokenization

In [10]:
# Initialisation du tokenizer BERT avec le modèle 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [11]:
# Prétraitement des données textuelles avec le tokenizer BERT
input_ids = []
attention_mask = []
for i in text:
    encoded_data = tokenizer.encode_plus(
    i,
    add_special_tokens=True,
    max_length=64,
    pad_to_max_length = True,
    return_attention_mask= True,
    return_tensors='pt')
    input_ids.append(encoded_data['input_ids'])
    attention_mask.append(encoded_data['attention_mask'])
input_ids = torch.cat(input_ids,dim=0)
attention_mask = torch.cat(attention_mask,dim=0)
# Conversion des étiquettes en tenseurs
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\Zacca\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


#### 3.2 Création de l'ensemble de données des ensembles d'entraînement, de test et de validation

In [12]:
# Création du dataset à partir des tensors d'input_ids, de masque d'attention et d'étiquettes
dataset = TensorDataset(input_ids, attention_mask, labels)

# Définition de la taille des ensembles d'entraînement et de validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Séparation aléatoire du dataset en ensembles d'entraînement et de validation
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Affichage de la taille de l'ensemble d'entraînement et de validation
print('Taille de l\'ensemble d\'entraînement -', train_size)
print('Taille de l\'ensemble de validation -', val_size)

Taille de l'ensemble d'entraînement - 16000
Taille de l'ensemble de validation - 4000


In [13]:
# Chargement des données d'entraînement avec le DataLoader
train_dl = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=32)

# Chargement des données de validation avec le DataLoader
val_dl = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=32)

#### 3.3 Preparation modele BERT

In [14]:
# Chargement du modèle BertForSequenceClassification à partir des poids pré-entraînés
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,  # Nombre de classes de sortie
    output_attentions=False,
    output_hidden_states=False
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Définition de l'optimiseur Adam avec un taux d'apprentissage de 2e-5 et epsilon de 1e-8
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# Définition du nombre d'époques
epochs = 1

# Calcul du nombre total d'étapes
total_steps = len(train_dl) * epochs

# Création du scheduler pour l'ajustement du taux d'apprentissage avec échauffement linéaire
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

#### 3.4 Entrainement modele BERT

In [17]:
# Start a new MLflow run
with mlflow.start_run(run_name='Bert Model'):

    # Libération de la mémoire GPU
    torch.cuda.empty_cache()

    # Boucle d'entraînement sur les époques
    for epoch in range(1, epochs+1):

        # Mettre le modèle en mode d'entraînement
        model.train()

        # Initialisation de la perte totale d'entraînement
        loss_train_total = 0

        for batch in train_dl:

            # Remise à zéro du gradient
            model.zero_grad()

            # Transfert du batch sur le périphérique GPU
            batch = tuple(b.to(device) for b in batch)

            # Extraction des entrées du batch
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         batch[2],
                     }

            # Propagation avant à travers le modèle
            outputs = model(**inputs)

            # Calcul de la perte
            loss = outputs[0]
            loss_train_total += loss.item()
            loss.backward()

            # Clip des gradients pour éviter l'explosion du gradient
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Mise à jour des poids du modèle
            optimizer.step()
            scheduler.step()

        # Affichage des informations à la fin de chaque époque
        print(f'\nEpoch {epoch}')

        # Calcul de la perte moyenne d'entraînement
        loss_train_avg = loss_train_total/len(train_dl)
        print(f'Training loss: {loss_train_avg}')

        # Log the training loss
        mlflow.log_metric("training_loss", loss_train_avg)

        # Évaluation sur l'ensemble de validation
        val_loss, predictions, true_vals = evaluate(val_dl)

        # Calcul de la précision sur l'ensemble de validation
        val_acc = accuracy(predictions, true_vals)

        # Log the validation loss and accuracy
        mlflow.log_metric("validation_loss", val_loss)
        mlflow.log_metric("accuracy", val_acc)

        # Affichage des résultats de validation
        print(f'Validation loss: {val_loss}')
        print(f'Accuracy: {val_acc}')

    # Log the model
    mlflow.pytorch.log_model(model, "model")


Epoch 1
Training loss: 0.4498463813662529
Validation loss: 0.4030902409553528
Accuracy: 0.82675


c:\Users\Zacca\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


### 4. Sauvegarde du modèle

In [18]:
# Répertoire de sortie pour enregistrer le modèle et le tokenizer
output_dir = '../model/BERT'

# Sauvegarde du modèle
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(output_dir)

# Sauvegarde du tokenizer
tokenizer.save_pretrained(output_dir)

('../model/BERT\\tokenizer_config.json',
 '../model/BERT\\special_tokens_map.json',
 '../model/BERT\\vocab.txt',
 '../model/BERT\\added_tokens.json')

In [19]:
# Chargement du tokenizer BERT
print('Chargement du tokenizer BERT...')
output_dir = '../model/BERT'
tokenizer = BertTokenizer.from_pretrained(output_dir)
model_loaded = BertForSequenceClassification.from_pretrained(output_dir)

Chargement du tokenizer BERT...


#### 5. Test du modèle sur un exemple

In [20]:
# test modele bert
test = Sentiment('it was a great movie')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\Zacca\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [21]:
# affichage resultat
if test == 1:
    print("Positive")
else:
    print("Negative")

Positive
